In [69]:
import geopandas as gpd
import pandas as pd

import numpy as np
from pathlib import Path
from datetime import datetime

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [44]:
df = pd.read_excel(io=Path(Path.cwd(), 'Задание 2.xlsx'))

df.shape

(53, 14)

# Проверка качества данныx. Приведение типов и форматов

In [89]:
df.head()

,longitude,latitude,customer,sales_to_customer,customer_sales,sales_to_customer_ABC,customers_profit,customers_profit_ABC,AOV,AOV_ABC,request_cnt_m,request_cnt_m2,request_cnt_m3,request_cnt_m_forecast
0,37.566022,55.744908,Клиент 1,50,70.0,NaN,10,NaN,150.0,NaN,50.0,65.00,84.5000,NaN
1,37.480038,55.707113,Клиент 2,70,98.0,NaN,24.5,NaN,300.0,NaN,4.0,2.80,1.9600,NaN
2,37.521722,55.675178,Клиент 3,50,70.0,NaN,35,NaN,500.0,NaN,3.0,300.00,30000.0000,NaN
3,37.612972,55.737492,Клиент 4,50,70.0,NaN,35,NaN,381.0,NaN,7.0,5.25,3.9375,NaN
4,37.447368,55.756777,Клиент 5,50,70.0,NaN,35,NaN,150.0,NaN,12.0,7.68,4.9152,NaN


In [51]:
df['sales_to_customer_ABC'] = np.nan
df['customers_profit_ABC'] = np.nan
df['AOV_ABC'] = np.nan
df['request_cnt_m_forecast'] = np.nan

In [46]:
df.columns = df.columns.str.lower()

In [47]:
df.columns.to_list()

['longitude',
 'latitude',
 'customer_group_4_id',
 'продажи клиенту',
 'продажи клиента',
 'группа клиента по продажам клиенту',
 'профит клиента',
 'группа клиентов по профиту',
 'средний чек',
 'группа клиентов по среднему чеку',
 'количество заявок в месяц',
 'количество заявок в месяц-1',
 'количество заявок в месяц-2',
 'количество заявок в месяц +1']

In [48]:
df = df.rename(columns={
    'customer_group_4_id': 'customer',
    'продажи клиенту': 'sales_to_customer',
    'продажи клиента': 'customer_sales',
    'группа клиента по продажам клиенту': 'sales_to_customer_ABC',
    'профит клиента': 'customers_profit',
    'группа клиентов по профиту': 'customers_profit_ABC',
    'средний чек': 'AOV',
    'группа клиентов по среднему чеку': 'AOV_ABC',
    'количество заявок в месяц': 'request_cnt_m',
    'количество заявок в месяц-1': 'request_cnt_m2',
    'количество заявок в месяц-2': 'request_cnt_m3',
    'количество заявок в месяц +1': 'request_cnt_m_forecast'
})

In [88]:
df = df.dropna(subset='customer')

In [57]:
df.dtypes

longitude                 float64
latitude                  float64
customer                   object
sales_to_customer          object
customer_sales            float64
sales_to_customer_ABC     float64
customers_profit           object
customers_profit_ABC      float64
AOV                       float64
AOV_ABC                   float64
request_cnt_m             float64
request_cnt_m2            float64
request_cnt_m3            float64
request_cnt_m_forecast    float64
dtype: object

In [49]:
df['longitude'] = df['longitude'].str.replace(',', '.')

df['latitude'] = df['latitude'].str.replace(',', '.')

In [56]:
df = df.astype({
    'latitude':'float64',
    'longitude':'float64',
})

In [86]:
df.loc[df.sales_to_customer.apply(lambda x: isinstance(x, datetime))==True, 'sales_to_customer'] = \
    df.loc[df.sales_to_customer.apply(lambda x: isinstance(x, datetime))==True]['sales_to_customer'].astype('datetime64').dt.day

In [92]:
df.sales_to_customer = df.sales_to_customer.astype(dtype='float64')

In [96]:
df.sales_to_customer = np.ceil(df.sales_to_customer)

df.sales_to_customer = df.sales_to_customer.astype('int64')

In [107]:
df.sales_to_customer.describe().reset_index().style.format(precision=0)

,index,sales_to_customer
0,count,52
1,mean,480835
2,std,3466867
3,min,1
4,25%,50
5,50%,70
6,75%,90
7,max,25000000


In [109]:
df.loc[df.sales_to_customer.quantile(q=0.99) < df.sales_to_customer, 'sales_to_customer'] = df.sales_to_customer.median()

In [112]:
df.customer_sales = np.ceil(df.customer_sales)

df.customer_sales = df.customer_sales.astype('int64')

In [114]:
df.customer_sales.describe()

count     52.000000
mean      99.384615
std       49.920389
min        2.000000
25%       70.000000
50%       98.000000
75%      126.000000
max      350.000000
Name: customer_sales, dtype: float64

In [115]:
df.loc[df.customers_profit.apply(lambda x: isinstance(x, datetime))==True, 'customers_profit'] = \
    df.loc[df.customers_profit.apply(lambda x: isinstance(x, datetime))==True]['customers_profit'].astype('datetime64').dt.day

In [117]:
df.customers_profit = np.ceil(df.customers_profit)

df.customers_profit = df.customers_profit.astype('int64')

In [119]:
df.customers_profit.describe()

count     52.000000
mean      27.250000
std       19.987128
min        1.000000
25%       14.000000
50%       18.000000
75%       35.000000
max      102.000000
Name: customers_profit, dtype: float64

In [121]:
df.AOV.describe()

count      52.000000
mean      551.719457
std       341.607136
min        50.000000
25%       222.000000
50%       517.925000
75%       770.111029
max      1500.000000
Name: AOV, dtype: float64

In [125]:
df.request_cnt_m = np.ceil(df.request_cnt_m)
df.request_cnt_m = df.request_cnt_m.astype('int64')

df.request_cnt_m.describe()

count    52.000000
mean     13.538462
std      13.367302
min       3.000000
25%       4.000000
50%       9.000000
75%      13.000000
max      50.000000
Name: request_cnt_m, dtype: float64

In [126]:
df.request_cnt_m2 = np.ceil(df.request_cnt_m2)
df.request_cnt_m2 = df.request_cnt_m2.astype('int64')

df.request_cnt_m2.describe()

count      52.000000
mean       38.884615
std       150.564513
min         1.000000
25%         3.000000
50%         7.000000
75%        17.000000
max      1060.000000
Name: request_cnt_m2, dtype: float64

In [127]:
df.request_cnt_m3 = np.ceil(df.request_cnt_m3)
df.request_cnt_m3 = df.request_cnt_m3.astype('int64')

df.request_cnt_m3.describe()

count       52.000000
mean      2328.480769
std      13074.699432
min          1.000000
25%          2.000000
50%          5.500000
75%         20.000000
max      90000.000000
Name: request_cnt_m3, dtype: float64

In [151]:
df.head(100)

,longitude,latitude,customer,sales_to_customer,customer_sales,sales_to_customer_ABC,customers_profit,customers_profit_ABC,AOV,AOV_ABC,request_cnt_m,request_cnt_m2,request_cnt_m3,request_cnt_m_forecast
13,37.546821,55.436228,Клиент 14,1,2,C,1,C,1500.000000,A,12,8,6,NaN
51,38.475900,55.293300,Клиент 52,5,7,C,4,C,1209.497059,A,7,6,4,NaN
50,36.204764,55.709589,Клиент 51,85,119,A,17,B,1190.250000,A,3,35,35,NaN
49,37.345833,55.735083,Клиент 50,50,70,B,18,B,1171.002941,A,13,10,7,NaN
42,37.582735,55.754566,Клиент 43,90,126,A,63,A,1036.273529,A,34,45,58,NaN
41,37.756825,55.459751,Клиент 42,90,126,A,63,A,1017.026471,A,9,2,1,NaN
40,37.712867,55.648397,Клиент 41,70,98,A,25,A,997.779412,A,7,5,3,NaN
39,37.200745,56.009542,Клиент 40,70,98,A,14,B,978.532353,A,3,3,2,NaN
32,37.290172,55.440506,Клиент 33,90,126,A,32,A,843.802941,A,50,50,50,NaN
31,36.783547,55.852102,Клиент 32,90,126,A,18,B,824.555882,A,3,2,2,NaN


# ABC-анализ

### Продажи клиенту

In [ ]:
df = df.sort_values(by=['sales_to_customer'], ascending=False)

df['perc_of_total'] = df['sales_to_customer'] / df['sales_to_customer'].sum(axis=0)

df['perc_of_total_cum'] = df['perc_of_total'].cumsum(axis=0)

df.loc[df['perc_of_total_cum'] <= 0.8, 'sales_to_customer_ABC'] = 'A'

df.loc[
    (df['perc_of_total_cum'] > 0.8)
    & (df['perc_of_total_cum'] <= 0.9), 
    'sales_to_customer_ABC'] = 'B'

df.loc[df['sales_to_customer_ABC'].isna()==True, 'sales_to_customer_ABC'] = 'C'

df = df.drop(columns=['perc_of_total', 'perc_of_total_cum'])

### Профит клиента

In [148]:
df = df.sort_values(by=['customers_profit'], ascending=False)

df['perc_of_total'] = df['customers_profit'] / df['customers_profit'].sum(axis=0)

df['perc_of_total_cum'] = df['perc_of_total'].cumsum(axis=0)

df.loc[df['perc_of_total_cum'] <= 0.8, 'customers_profit_ABC'] = 'A'

df.loc[
    (df['perc_of_total_cum'] > 0.8)
    & (df['perc_of_total_cum'] <= 0.9), 
    'customers_profit_ABC'] = 'B'

df.loc[df['customers_profit_ABC'].isna()==True, 'customers_profit_ABC'] = 'C'

df = df.drop(columns=['perc_of_total', 'perc_of_total_cum'])

### AOV

In [150]:
df = df.sort_values(by=['AOV'], ascending=False)

df['perc_of_total'] = df['AOV'] / df['AOV'].sum(axis=0)

df['perc_of_total_cum'] = df['perc_of_total'].cumsum(axis=0)

df.loc[df['perc_of_total_cum'] <= 0.8, 'AOV_ABC'] = 'A'

df.loc[
    (df['perc_of_total_cum'] > 0.8)
    & (df['perc_of_total_cum'] <= 0.9), 
    'AOV_ABC'] = 'B'

df.loc[df['AOV_ABC'].isna()==True, 'AOV_ABC'] = 'C'

df = df.drop(columns=['perc_of_total', 'perc_of_total_cum'])

# Forecast

In [180]:
df['request_cnt_m_forecast'] = df['request_cnt_m_forecast'].astype('int64')

In [181]:
df.head(100)

,longitude,latitude,customer,sales_to_customer,customer_sales,sales_to_customer_ABC,customers_profit,customers_profit_ABC,AOV,AOV_ABC,request_cnt_m,request_cnt_m2,request_cnt_m3,request_cnt_m_forecast,m_m2,m2_m3,coeff
13,37.546821,55.436228,Клиент 14,1,2,C,1,C,1500.000000,A,12,8,6,2,6.0,5.0,5.5
51,38.475900,55.293300,Клиент 52,5,7,C,4,C,1209.497059,A,7,6,4,1,6.0,3.0,4.5
50,36.204764,55.709589,Клиент 51,85,119,A,17,B,1190.250000,A,3,35,35,1,409.0,35.0,222.0
49,37.345833,55.735083,Клиент 50,50,70,B,18,B,1171.002941,A,13,10,7,2,8.0,5.0,6.5
42,37.582735,55.754566,Клиент 43,90,126,A,63,A,1036.273529,A,34,45,58,1,60.0,75.0,67.5
41,37.756825,55.459751,Клиент 42,90,126,A,63,A,1017.026471,A,9,2,1,1,1.0,1.0,1.0
40,37.712867,55.648397,Клиент 41,70,98,A,25,A,997.779412,A,7,5,3,1,4.0,2.0,3.0
39,37.200745,56.009542,Клиент 40,70,98,A,14,B,978.532353,A,3,3,2,1,3.0,2.0,2.5
32,37.290172,55.440506,Клиент 33,90,126,A,32,A,843.802941,A,50,50,50,1,50.0,50.0,50.0
31,36.783547,55.852102,Клиент 32,90,126,A,18,B,824.555882,A,3,2,2,1,2.0,2.0,2.0


In [167]:
df['m_m2'] = np.ceil(df['request_cnt_m2'] / (df['request_cnt_m'] / df['request_cnt_m2']))

In [168]:
df['m2_m3'] =  np.ceil(df['request_cnt_m3'] / (df['request_cnt_m2'] / df['request_cnt_m3']))

In [171]:
df['coeff'] = df[['m_m2', 'm2_m3']].mean(axis=1)

In [179]:
df['request_cnt_m_forecast'] = np.ceil(df['request_cnt_m3'] / df['coeff'])

# Lheu

In [14]:
df = df[['customer', 'latitude', 'longitude']].merge(df[['customer', 'latitude', 'longitude']], how='cross')

df = df[df['customer_x']!=df['customer_y']]

In [15]:
df.head()

,customer_x,latitude_x,longitude_x,customer_y,latitude_y,longitude_y
1,Клиент 1,55.744908,37.566022,Клиент 2,55.707113,37.480038
2,Клиент 1,55.744908,37.566022,Клиент 3,55.675178,37.521722
3,Клиент 1,55.744908,37.566022,Клиент 4,55.737492,37.612972
4,Клиент 1,55.744908,37.566022,Клиент 5,55.756777,37.447368
5,Клиент 1,55.744908,37.566022,Клиент 6,55.771022,37.197169


In [16]:
df1 = df[['customer_x', 'latitude_x', 'longitude_x']]

df2 = df[['customer_y', 'latitude_y', 'longitude_y']]

In [17]:
df1 = df1.rename(columns={
    'customer_x': 'customer',
    'latitude_x': 'latitude',
    'longitude_x': 'longitude'
})

df2 = df2.rename(columns={
    'customer_y': 'customer',
    'latitude_y': 'latitude',
    'longitude_y': 'longitude'
})

In [18]:
gdf1 = gpd.GeoDataFrame(
    data=df1,
    geometry=gpd.points_from_xy(
        x=df1["longitude"], y=df1["latitude"],
        crs='EPSG:4326'
    ),
)

gdf2 = gpd.GeoDataFrame(
    data=df2,
    geometry=gpd.points_from_xy(
        x=df2["longitude"], y=df2["latitude"],
        crs='EPSG:4326'
    ),
)

In [20]:
gdf1.geometry = gdf1.geometry.to_crs(epsg=3857)

gdf2.geometry = gdf2.geometry.to_crs(epsg=3857)

In [21]:
gdf1.head()

,customer,latitude,longitude,geometry
1,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
2,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
3,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
4,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
5,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)


In [25]:
distance = gdf1.distance(gdf2).reset_index()

distance = distance.rename(columns={0: 'distance'})

In [29]:
distance.head()

,index,distance
0,1,12142.228384
1,2,14634.024237
2,3,5428.299253
3,4,13415.517013
4,5,41384.261630


In [27]:
gdf1.head()

,customer,latitude,longitude,geometry
1,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
2,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
3,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
4,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)
5,Клиент 1,55.744908,37.566022,POINT (4181830.440 7507800.749)


In [30]:
distance.shape

(2757, 2)

In [31]:
gdf1.shape

(2757, 4)

In [34]:
gdf2.head()

,customer,latitude,longitude,geometry
1,Клиент 2,55.707113,37.480038,POINT (4172258.745 7500329.715)
2,Клиент 3,55.675178,37.521722,POINT (4176898.987 7494022.672)
3,Клиент 4,55.737492,37.612972,POINT (4187056.890 7506334.239)
4,Клиент 5,55.756777,37.447368,POINT (4168621.937 7510148.418)
5,Клиент 6,55.771022,37.197169,POINT (4140769.912 7512967.000)


In [38]:
test = gdf1.merge(distance, how='left', left_index=True, right_on='index').merge(gdf2[['customer']], how='left', left_on='index', right_index=True)

In [42]:
test[test['distance']<=2000].head(1000)

,customer_x,latitude,longitude,geometry,index,distance,customer_y
483,Клиент 10,55.792342,37.780844,POINT (4205744.316 7517187.398),493,445.015231,Клиент 17
530,Клиент 11,55.748275,37.625733,POINT (4188477.438 7508466.664),541,0.000000,Клиент 12
531,Клиент 11,55.748275,37.625733,POINT (4188477.438 7508466.664),542,0.000000,Клиент 13
582,Клиент 12,55.748275,37.625733,POINT (4188477.438 7508466.664),593,0.000000,Клиент 11
583,Клиент 12,55.748275,37.625733,POINT (4188477.438 7508466.664),595,0.000000,Клиент 13
634,Клиент 13,55.748275,37.625733,POINT (4188477.438 7508466.664),646,0.000000,Клиент 11
635,Клиент 13,55.748275,37.625733,POINT (4188477.438 7508466.664),647,0.000000,Клиент 12
797,Клиент 16,55.878046,37.542802,POINT (4179245.602 7534176.271),813,1804.083715,Клиент 19
841,Клиент 17,55.794214,37.783056,POINT (4205990.555 7517558.080),857,445.015231,Клиент 10
951,Клиент 19,55.869106,37.539850,POINT (4178916.986 7532402.368),969,1804.083715,Клиент 16


# Prev version

In [114]:
gdf.geometry = gdf.geometry.to_crs(epsg=3857)

In [115]:
gdf['point_polygon'] = gdf['geometry'].buffer(2000)

In [116]:
gdf1 = gdf[['customer', 'point_polygon']]

gdf1 = gdf1.rename(columns={'point_polygon':'geometry'})

gdf1 = gdf1.set_geometry('geometry')

In [117]:
gdf2 = gdf[['customer', 'geometry']]

In [118]:
test = gpd.sjoin(left_df=gdf2, right_df=gdf1, how='inner', predicate='intersects')

test.shape

(66, 4)

In [119]:
test = test[test['customer_left']!=test['customer_right']]

In [120]:
test = test.drop_duplicates(subset=['customer_left'])

test

,customer_left,geometry,index_right,customer_right
16,Клиент 17,POINT (4205990.555 7517558.080),9,Клиент 10
9,Клиент 10,POINT (4205744.316 7517187.398),16,Клиент 17
10,Клиент 11,POINT (4188477.438 7508466.664),12,Клиент 13
11,Клиент 12,POINT (4188477.438 7508466.664),12,Клиент 13
12,Клиент 13,POINT (4188477.438 7508466.664),10,Клиент 11
15,Клиент 16,POINT (4179245.602 7534176.271),18,Клиент 19
18,Клиент 19,POINT (4178916.986 7532402.368),15,Клиент 16
34,Клиент 35,POINT (4171674.986 7333791.904),35,Клиент 36
35,Клиент 36,POINT (4171674.986 7333791.904),34,Клиент 35
46,Клиент 47,POINT (4186334.315 7478431.525),47,Клиент 48


In [121]:
test['rank'] = test.geometry.to_wkt().rank(method='dense')

In [122]:
test.sort_values(by='rank').head(100)

,customer_left,geometry,index_right,customer_right,rank
34,Клиент 35,POINT (4171674.986 7333791.904),35,Клиент 36,1.0
35,Клиент 36,POINT (4171674.986 7333791.904),34,Клиент 35,1.0
18,Клиент 19,POINT (4178916.986 7532402.368),15,Клиент 16,2.0
15,Клиент 16,POINT (4179245.602 7534176.271),18,Клиент 19,3.0
46,Клиент 47,POINT (4186334.315 7478431.525),47,Клиент 48,4.0
47,Клиент 48,POINT (4186334.315 7478431.525),46,Клиент 47,4.0
10,Клиент 11,POINT (4188477.438 7508466.664),12,Клиент 13,5.0
11,Клиент 12,POINT (4188477.438 7508466.664),12,Клиент 13,5.0
12,Клиент 13,POINT (4188477.438 7508466.664),10,Клиент 11,5.0
9,Клиент 10,POINT (4205744.316 7517187.398),16,Клиент 17,6.0


In [124]:
gdf1.head()

,customer,geometry
0,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810..."
1,Клиент 2,"POLYGON ((4174258.745 7500329.715, 4174249.115..."
2,Клиент 3,"POLYGON ((4178898.987 7494022.672, 4178889.356..."
3,Клиент 4,"POLYGON ((4189056.890 7506334.239, 4189047.260..."
4,Клиент 5,"POLYGON ((4170621.937 7510148.418, 4170612.307..."


In [125]:
gdf2.head()

,customer,geometry
0,Клиент 1,POINT (4181830.440 7507800.749)
1,Клиент 2,POINT (4172258.745 7500329.715)
2,Клиент 3,POINT (4176898.987 7494022.672)
3,Клиент 4,POINT (4187056.890 7506334.239)
4,Клиент 5,POINT (4168621.937 7510148.418)


In [126]:
gdf_merged = gdf1.merge(gdf2, how='cross')

In [128]:
gdf_merged.shape

(2809, 4)

In [129]:
gdf_merged = gdf_merged[gdf_merged['customer_x']!=gdf_merged['customer_y']]

gdf_merged.shape

(2757, 4)

In [140]:
gdf_merged.head()

,customer_x,geometry_x,customer_y,geometry_y
1,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810...",Клиент 2,POINT (4172258.745 7500329.715)
2,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810...",Клиент 3,POINT (4176898.987 7494022.672)
3,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810...",Клиент 4,POINT (4187056.890 7506334.239)
4,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810...",Клиент 5,POINT (4168621.937 7510148.418)
5,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810...",Клиент 6,POINT (4140769.912 7512967.000)


In [132]:
gdf_merged1 = gdf_merged[['customer_x', 'geometry_x']]

gdf_merged2 = gdf_merged[['customer_y','geometry_y']]

In [134]:
gdf_merged1 = gdf_merged1.set_geometry('geometry_x')

gdf_merged2 = gdf_merged2.set_geometry('geometry_y')

In [148]:
gdf_merged1.head()

,customer_x,geometry_x
1,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810..."
2,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810..."
3,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810..."
4,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810..."
5,Клиент 1,"POLYGON ((4183830.440 7507800.749, 4183820.810..."


In [149]:
gdf_merged2.head()

,customer_y,geometry_y
1,Клиент 2,POINT (4172258.745 7500329.715)
2,Клиент 3,POINT (4176898.987 7494022.672)
3,Клиент 4,POINT (4187056.890 7506334.239)
4,Клиент 5,POINT (4168621.937 7510148.418)
5,Клиент 6,POINT (4140769.912 7512967.000)


In [136]:
test = gpd.sjoin(gdf_merged1, gdf_merged2, how='inner', predicate='intersects')

In [141]:
test = test[test['customer_x']!=test['customer_y']]

In [144]:
test = test.drop_duplicates(subset=['customer_x','customer_y'])

In [147]:
test.head(100)

,customer_x,geometry_x,index_right,customer_y
848,Клиент 17,"POLYGON ((4207990.555 7517558.080, 4207980.924...",115,Клиент 10
477,Клиент 10,"POLYGON ((4207744.316 7517187.398, 4207734.685...",1765,Клиент 17
583,Клиент 12,"POLYGON ((4190477.438 7508466.664, 4190467.808...",2077,Клиент 11
636,Клиент 13,"POLYGON ((4190477.438 7508466.664, 4190467.808...",2077,Клиент 11
530,Клиент 11,"POLYGON ((4190477.438 7508466.664, 4190467.808...",1072,Клиент 13
583,Клиент 12,"POLYGON ((4190477.438 7508466.664, 4190467.808...",1072,Клиент 13
530,Клиент 11,"POLYGON ((4190477.438 7508466.664, 4190467.808...",2078,Клиент 12
636,Клиент 13,"POLYGON ((4190477.438 7508466.664, 4190467.808...",2078,Клиент 12
795,Клиент 16,"POLYGON ((4181245.602 7534176.271, 4181235.971...",654,Клиент 19
954,Клиент 19,"POLYGON ((4180916.986 7532402.368, 4180907.356...",863,Клиент 16
